In [1]:
import pandas as pd
import requests
import sys
import time

noaa_key = "JOwIFUXjlWBqtstSqsSaKYDMzShAccQe"
usda_key = "8D5EDCA6-733D-3ACA-AF06-BA5BB3938B4F"

In [2]:
# API endpoint
url = "https://quickstats.nass.usda.gov/api/api_GET/"

params = {
    "key": usda_key,
    "commodity_desc": "CORN",
    "statisticcat_desc": "YIELD",
    "unit_desc": "BU / ACRE",
    "state_alpha": "MI",
    "year": "2022",
    "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
    "agg_level_desc": "COUNTY",
    "format": "JSON"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print(data)  # or process the data as needed
else:
    print(f"Request failed with status code {response.status_code}")

{'data': [{'util_practice_desc': 'GRAIN', 'domaincat_desc': 'NOT SPECIFIED', 'begin_code': '00', 'state_ansi': '26', 'sector_desc': 'CROPS', 'county_name': 'DELTA', 'load_time': '2024-02-23 15:00:00.000', 'statisticcat_desc': 'YIELD', 'commodity_desc': 'CORN', 'watershed_code': '00000000', 'week_ending': '', 'location_desc': 'MICHIGAN, UPPER PENINSULA, DELTA', 'source_desc': 'SURVEY', 'county_ansi': '041', 'county_code': '041', 'class_desc': 'ALL CLASSES', 'unit_desc': 'BU / ACRE', 'CV (%)': '12.8', 'watershed_desc': '', 'asd_code': '10', 'country_code': '9000', 'short_desc': 'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE', 'freq_desc': 'ANNUAL', 'reference_period_desc': 'YEAR', 'congr_district_code': '', 'group_desc': 'FIELD CROPS', 'country_name': 'UNITED STATES', 'state_alpha': 'MI', 'region_desc': '', 'state_fips_code': '26', 'prodn_practice_desc': 'ALL PRODUCTION PRACTICES', 'asd_desc': 'UPPER PENINSULA', 'year': 2022, 'agg_level_desc': 'COUNTY', 'zip_5': '', 'end_code': '00', 'domai

In [3]:
pd.set_option('display.max_rows', 10) 
usda_df = pd.DataFrame(data['data'])
usda_df

,util_practice_desc,domaincat_desc,begin_code,state_ansi,sector_desc,county_name,load_time,statisticcat_desc,commodity_desc,watershed_code,...,state_fips_code,prodn_practice_desc,asd_desc,year,agg_level_desc,zip_5,end_code,domain_desc,Value,state_name
0,GRAIN,NOT SPECIFIED,00,26,CROPS,DELTA,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,UPPER PENINSULA,2022,COUNTY,,00,TOTAL,89.7,MICHIGAN
1,GRAIN,NOT SPECIFIED,00,26,CROPS,DICKINSON,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,UPPER PENINSULA,2022,COUNTY,,00,TOTAL,108.9,MICHIGAN
2,GRAIN,NOT SPECIFIED,00,26,CROPS,MENOMINEE,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,UPPER PENINSULA,2022,COUNTY,,00,TOTAL,91.3,MICHIGAN
3,GRAIN,NOT SPECIFIED,00,26,CROPS,CHARLEVOIX,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,NORTHWEST,2022,COUNTY,,00,TOTAL,112.2,MICHIGAN
4,GRAIN,NOT SPECIFIED,00,26,CROPS,EMMET,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,NORTHWEST,2022,COUNTY,,00,TOTAL,106,MICHIGAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,GRAIN,NOT SPECIFIED,00,26,CROPS,OAKLAND,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,SOUTHEAST,2022,COUNTY,,00,TOTAL,142.6,MICHIGAN
56,GRAIN,NOT SPECIFIED,00,26,CROPS,ST CLAIR,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,SOUTHEAST,2022,COUNTY,,00,TOTAL,146.7,MICHIGAN
57,GRAIN,NOT SPECIFIED,00,26,CROPS,WASHTENAW,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,SOUTHEAST,2022,COUNTY,,00,TOTAL,165.8,MICHIGAN
58,GRAIN,NOT SPECIFIED,00,26,CROPS,WAYNE,2024-02-23 15:00:00.000,YIELD,CORN,00000000,...,26,ALL PRODUCTION PRACTICES,SOUTHEAST,2022,COUNTY,,00,TOTAL,120.4,MICHIGAN


In [4]:
def is_valid_data(data):
  return data and isinstance(data, dict) and "results" in data and data["results"]

# Sate in USDA data is state_ansi
def get_raw_weather(state_ansi, county_ansi, startdate, enddate):
  url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

  headers = {
  'token': noaa_key
  }

  # Convert startdate and enddate to datetime objects
  startdate = pd.to_datetime(startdate)
  enddate = pd.to_datetime(enddate)

  # Calculate the number of days between startdate and enddate
  num_days = (enddate - startdate).days

  # Check if the number of days is greater than 30
  if num_days > 30:
    # Split the date range into multiple requests
    requests_list = []
    current_date = startdate
    while current_date <= enddate:
      # Calculate the end date for the current request
      current_enddate = current_date + pd.DateOffset(days=30)
      if current_enddate > enddate:
        current_enddate = enddate

      # Create the parameters for the current request
      params = {
        "datasetid": "GHCND",
        "startdate": current_date.strftime("%Y-%m-%d"),
        "enddate": current_enddate.strftime("%Y-%m-%d"),
        "units": "standard",
        "limit": "1000", # 1000 is the max limit
        "locationid": f"FIPS:{state_ansi}{county_ansi}",
        'datatypeid': ['TMAX', 'TMIN']
      }

      # Make the request
      print("FIPS:{state_ansi}{county_ansi}")
      response = requests.get(url, headers=headers, params=params)

      if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
          # Append the results to the list
          requests_list.append(pd.DataFrame(data["results"]))
        else:
          print(f"No data found for FIPS:{params['locationid']}")
      else:
        print(response.text)

      # Move to the next date range
      current_date = current_enddate + pd.DateOffset(days=1)

    # Concatenate the results from all requests
    if requests_list:
      return pd.concat(requests_list)
    else:
      return None
  else:
    # Single request for the entire date range
    params = {
      "datasetid": "GHCND",
      "startdate": startdate.strftime("%Y-%m-%d"),
      "enddate": enddate.strftime("%Y-%m-%d"),
      "units": "standard",
      "limit": "1000",
      "locationid": f"FIPS:{state_ansi}{county_ansi}",
      'datatypeid': ['TMAX', 'TMIN']
    }
    print(f"FIPS:{state_ansi}{county_ansi}")
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
      data = response.json()
      # Return data as a DataFrame
      print(data)
      # Check if 'results' key is present in the response
      if is_valid_data(data):
        return pd.DataFrame(data["results"])
      else:
        print(f"No data found for FIPS:{params['locationid']}")
        return None
    else:
      print(response.text)
      return None

In [5]:
weather_raw = get_raw_weather("26", "021", "2022-12-20T00:00:00", "2022-12-31T00:00:00")

# Save the raw weather data to a CSV file
weather_raw.to_csv("weather_raw.csv", index=False)

FIPS:26021
{'metadata': {'resultset': {'offset': 1, 'count': 72, 'limit': 1000}}, 'results': [{'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00200710', 'attributes': ',,7,1600', 'value': 29.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00200710', 'attributes': ',,7,1600', 'value': 21.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00205892', 'attributes': ',,7,1500', 'value': 35.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00205892', 'attributes': ',,7,1500', 'value': 23.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USW00094871', 'attributes': ',,W,2400', 'value': 34.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USW00094871', 'attributes': ',,W,2400', 'value': 22.0}, {'date': '2022-12-21T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00200710', 'attributes': ',,7,1600', 'value': 32.0}, {'date': '2022-12-21T00:

In [6]:
def clean_weather(df,state_ansi,county_ansi):
    # For each day in date column average the TMAX and TMIN into their own columns across all stations
    df["date"] = pd.to_datetime(df["date"])
    df["TMAX"] = df["value"].where(df["datatype"] == "TMAX")
    df["TMIN"] = df["value"].where(df["datatype"] == "TMIN")
    
    df = df.groupby("date").agg({"TMAX": "mean", "TMIN": "mean"}).reset_index()

    # add state and county columns
    df["state"] = state_ansi
    df["county"] = county_ansi
    
    return df

In [7]:
weather = clean_weather(weather_raw, "26", "145")

# Print the first 25 rows of the weather DataFrame
weather.head(25)
weather.to_csv("weather.csv", index=False)

In [8]:
# for each state_ansi county_ansi combo in the USDA data, get the weather data

def get_weather_data(usda_df):
    weather_data = []
    for index, row in usda_df.iterrows():
        state_ansi = row["state_ansi"]
        county_ansi = row["county_ansi"]
        time.sleep(1)
        weather_raw = get_raw_weather(state_ansi, county_ansi, "2022-10-20T00:00:00", "2022-12-31T00:00:00")
        if weather_raw is not None:
            weather = clean_weather(weather_raw, state_ansi, county_ansi)
            weather_data.append(weather)
    return pd.concat(weather_data)

weather_data = get_weather_data(usda_df)
weather_data.to_csv("weather_data.csv", index=False)

FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
No data found for FIPS:FIPS:26109
FIPS:{state_ansi}{county_ansi}
No data found for FIPS:FIPS:26109
FIPS:{state_ansi}{county_ansi}
No data found for FIPS:FIPS:26109
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}
FIPS:{state_ansi}{county_ansi}

In [9]:
# Count the number of unique items in column county for weather_data
weather_data["county"].nunique()

56